In [1]:
import numpy as np
import time
import random
import csv
start_time = time.time()

In [2]:
distances = np.array([
    [0, 11, 11, 14, 16, 16, 12, 14, 14, 19, 20],
    [11, 0, 1, 3, 5, 5, 2, 4, 4, 9, 10],
    [11, 1, 0, 4, 6, 6, 1, 3, 3, 8, 9],
    [14, 3, 4, 0, 2, 2, 5, 7, 7, 12, 13],
    [16, 5, 6, 2, 0, 2, 6, 8, 8, 10, 9],
    [16, 5, 6, 2, 2, 0, 4, 6, 6, 8, 7],
    [12, 2, 1, 5, 6, 4, 0, 2, 2, 7, 8],
    [14, 4, 3, 7, 8, 6, 2, 0, 1, 6, 7],
    [14, 4, 3, 7, 8, 6, 2, 1, 0, 5, 6],
    [19, 9, 8, 12, 10, 8, 7, 6, 5, 0, 1],
    [20, 10, 9, 13, 9, 7, 8, 7, 6, 1, 0]
])

num_vehicles = 31
vehicle_capacity = 5
demand = [0, 77, 77, 153, 77, 77, 77, 37, 37, 37, 37]
num_locations = len(demand)

In [3]:
def calculate_route_cost(route, distances):
    cost = 0
    for i in range(len(route) - 1):
        cost += distances[route[i]][route[i + 1]]
    return cost
def update_demand(route, demand):
    for node in route[1:-1]:
        demand[node] = max(0, demand[node] - vehicle_capacity)

In [4]:
def solve_vrp_lns(distances, demand, num_vehicles, vehicle_capacity, max_iter=100):
    remaining_demand = demand.copy()
    routes = []
    total_distance = 0
    vehicle_trip_counters = {i: 0 for i in range(1, num_vehicles + 1)}
    while any(d > 0 for d in remaining_demand):
        for vehicle in range(1, num_vehicles + 1):
            if all(d == 0 for d in remaining_demand):
                break
            current_demand = remaining_demand.copy()
            current_route = [0]
            current_load = 0
            nodes = [i for i in range(1, len(current_demand)) if current_demand[i] > 0]
            random.shuffle(nodes)
            while current_load < vehicle_capacity and nodes:
                next_node = None
                min_distance = float('inf')
                for node in nodes:
                    if distances[current_route[-1]][node] < min_distance:
                        next_node = node
                        min_distance = distances[current_route[-1]][node]
                if next_node is None:
                    break
                load = min(vehicle_capacity - current_load, current_demand[next_node])
                if load == 0:
                    break
                current_route.append(next_node)
                current_load += load
                current_demand[next_node] -= load
                nodes.remove(next_node)
            current_route.append(0)
            vehicle_trip_counters[vehicle] += 1
            vehicle_number = f"{vehicle}.{vehicle_trip_counters[vehicle]}"
            routes.append((vehicle_number, current_route))
            total_distance += calculate_route_cost(current_route, distances)
            update_demand(current_route, remaining_demand)
    return routes, total_distance, remaining_demand
routes, total_distance, remaining_demand = solve_vrp_lns(distances, demand, num_vehicles, vehicle_capacity)

In [5]:
print("Routes:")
route_dict = {}
for vehicle_number, route in routes:
    route_tuple = tuple(route)
    if route_tuple not in route_dict:
        route_dict[route_tuple] = []
    route_dict[route_tuple].append(vehicle_number)
for route, vehicles in route_dict.items():
    distance = calculate_route_cost(route, distances)
    vehicles_str = ", ".join(vehicles)
    print(f"Route: {route}")
    print(f"Vehicles: {vehicles_str}")
    print(f"Distance Traveled: {distance}")
    print("-" * 40)
csv_filename = "vrp_routes_results.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Route", "Vehicles", "Distance Traveled"])  # Write header
    for route, vehicles in route_dict.items():
        distance = calculate_route_cost(route, distances)
        vehicles_str = ", ".join(vehicles)  # Combine vehicle numbers for identical routes
        writer.writerow([route, vehicles_str, distance])

Routes:
Route: (0, 2, 0)
Vehicles: 1.1, 3.1, 4.1, 5.1, 7.1, 11.1, 12.1, 16.1, 18.1, 19.1, 22.1, 24.1, 25.1, 26.1, 27.1
Distance Traveled: 22
----------------------------------------
Route: (0, 1, 0)
Vehicles: 2.1, 6.1, 8.1, 9.1, 10.1, 13.1, 14.1, 15.1, 17.1, 20.1, 21.1, 23.1, 28.1, 30.1, 31.1
Distance Traveled: 22
----------------------------------------
Route: (0, 2, 6, 0)
Vehicles: 29.1
Distance Traveled: 24
----------------------------------------
Route: (0, 1, 6, 0)
Vehicles: 1.2
Distance Traveled: 25
----------------------------------------
Route: (0, 6, 0)
Vehicles: 2.2, 3.2, 4.2, 5.2, 6.2, 7.2, 8.2, 9.2, 10.2, 11.2, 12.2, 13.2, 14.2
Distance Traveled: 24
----------------------------------------
Route: (0, 6, 8, 0)
Vehicles: 15.2
Distance Traveled: 28
----------------------------------------
Route: (0, 3, 0)
Vehicles: 16.2, 17.2, 18.2, 22.2, 23.2, 25.2, 29.2, 30.2, 3.3, 7.3, 8.3, 9.3, 11.3, 12.3, 13.3, 14.3, 15.3, 16.3, 17.3, 18.3, 19.3, 20.3, 21.3, 22.3, 23.3, 24.3, 25.3, 26.3, 

In [6]:
vehicle_summary = {}
for vehicle_number, route in routes:
    if vehicle_number not in vehicle_summary:
        vehicle_summary[vehicle_number] = {
            "routes": [],
            "distance": 0
        }
    route_distance = calculate_route_cost(route, distances)
    vehicle_summary[vehicle_number]["routes"].append(route)
    vehicle_summary[vehicle_number]["distance"] += route_distance
print("\nVehicle Summary:")
for vehicle, details in vehicle_summary.items():
    print(f"Vehicle: {vehicle}")
    print(f"  Routes: {details['routes']}")
    print(f"  Total Distance: {details['distance']}")
    print("-" * 40)
csv_vehicle_summary_filename = "vrp_vehicle_summary.csv"
with open(csv_vehicle_summary_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Vehicle", "Routes", "Total Distance"])
    for vehicle, details in vehicle_summary.items():
        routes_str = "; ".join(map(str, details["routes"]))
        writer.writerow([vehicle, routes_str, details["distance"]])
print(f"\nResults exported to {csv_filename}")
print(f"Total Distance: {total_distance}")
print(f"Remaining Demand: {remaining_demand}")
print(f"\nVehicle summary exported to {csv_vehicle_summary_filename}")
end_time = time.time()


Vehicle Summary:
Vehicle: 1.1
  Routes: [[0, 2, 0]]
  Total Distance: 22
----------------------------------------
Vehicle: 2.1
  Routes: [[0, 1, 0]]
  Total Distance: 22
----------------------------------------
Vehicle: 3.1
  Routes: [[0, 2, 0]]
  Total Distance: 22
----------------------------------------
Vehicle: 4.1
  Routes: [[0, 2, 0]]
  Total Distance: 22
----------------------------------------
Vehicle: 5.1
  Routes: [[0, 2, 0]]
  Total Distance: 22
----------------------------------------
Vehicle: 6.1
  Routes: [[0, 1, 0]]
  Total Distance: 22
----------------------------------------
Vehicle: 7.1
  Routes: [[0, 2, 0]]
  Total Distance: 22
----------------------------------------
Vehicle: 8.1
  Routes: [[0, 1, 0]]
  Total Distance: 22
----------------------------------------
Vehicle: 9.1
  Routes: [[0, 1, 0]]
  Total Distance: 22
----------------------------------------
Vehicle: 10.1
  Routes: [[0, 1, 0]]
  Total Distance: 22
----------------------------------------
Vehicle: 11

In [7]:
print(f"Execution Time: {end_time - start_time:.2f} seconds")

Execution Time: 0.10 seconds
